Testing SuperK SELECT control via Python. We are using NKT Photonics SuperK Compact laser with SELECT tunable filter. Laser is on COM3.

Python interface doesn't work while Control software is connected.

In [3]:
from controls import *

Loading x64 DLL from: C:\NKTP_SDK\NKTPDLL\x64\NKTPDLL.dll


FileNotFoundError: Could not find module 'C:\NKTP_SDK\NKTPDLL\x64\NKTPDLL.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [2]:
from NKTP_DLL import *
from controls import scan_ports, compact, select, driver

Loading x64 DLL from: C:\NKTP_SDK\NKTPDLL\x64\NKTPDLL.dll


FileNotFoundError: Could not find module 'C:\NKTP_SDK\NKTPDLL\x64\NKTPDLL.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [6]:
scan_ports()

Following ports has modules: COM3
Comport: COM3 Device type: 0x0074 at address: 1
Comport: COM3 Device type: 0x0067 at address: 16
Comport: COM3 Device type: 0x0066 at address: 17
Close result:  0:OPSuccess


In [ ]:
compact.get_interlock()

Waiting for interlock to reset.


In [ ]:
compact.reset_interlock()

Interlock is OK.


In [ ]:
compact.trig_mode()

Laser mode: Internal frequency generator


In [ ]:
compact.emission_on()

Setting emission ON. 0:RegResultSuccess


In [ ]:
compact.overall_power(20)

Setting overall power level to 20%.


In [ ]:
compact.emission_off()

Setting emission OFF. 0:RegResultSuccess


"All wavelength and amplitude settings are sent to the RF Driver, not to the SuperK SELECT"
First channel is controlled with 0x90, second with 0x91, and so on up to 0x97 for the eighth channel. 
Only use the element on index 0. Can send only one 32 bit integer to change this value. 

In [108]:
COMport = 'COM3' # depends on the port the device is connected to
COMPACT_devID = 1 # fixed for the SuperK COMPACT
SELECT_devID = 16 # fixed for the SuperK SELECT
RFdriver_devID = 17 # fixed for the SuperK RF driver

In [ ]:
def set_channel(channel, wavelength, amplitude):          # WORKS
    """Specify the channel (1-8), then set the wavelength in nm and amplitude as a percent for that channel."""
    wavelength_channel_ID = int(channel-1)
    wavelength_address = f"0x9{wavelength_channel_ID}"
    wavelength_address = int(wavelength_address, 16)
    wavelength_pm = int(wavelength*1000) # convert nm to pm as the input format

    amplitude_channel_ID = int(channel-1)
    amplitude_address = f"0xB{amplitude_channel_ID}"
    amplitude_address = int(amplitude_address, 16)
    amplitude_percent = int(amplitude*10) # convert to tenths of a percent as the imput format

    result1 = registerWriteReadU32(COMport, RFdriver_devID, wavelength_address, wavelength_pm, 0) # send index 0 if not in FSK mode
    result2 = registerWriteReadU16(COMport, RFdriver_devID, amplitude_address, amplitude_percent, -1)

    print(f'Channel {channel} wavelength set to:', result1[1]/1000, 'nm, amplitude set to:', result2[1]/10, '%.')

In [ ]:
driver.RF_power_off()

RF power is off.


In [ ]:
compact.emission_off()

Setting emission OFF. 0:RegResultSuccess


In [ ]:
driver.set_channel(1, 1800, 20)

Channel 1 wavelength set to: 1800.0 nm, amplitude set to: 20.0 %.


In [ ]:
driver.set_channel(2, 1400, 0)

Channel 2 wavelength set to: 1400.0 nm, amplitude set to: 0.0 %.


In [ ]:
driver.set_channel(3, 1820, 0)

Channel 3 wavelength set to: 1820.0 nm, amplitude set to: 0.0 %.


In [ ]:
driver.RF_power_on()

RF power is on.


TODO: double check the device address for RF driver

In [8]:
# TRY GETTING A RESPONSE FROM THE DIFFERENT RESULT TYPES

def wavelength_min1():
    """Crystal 1 (VIS/NIR) minimum wavelength in nm."""
    result = registerReadU32(COMport, RFdriver_devID, 0x90, -1)
    wavelength = result[1]/1000 # convert pm to nm
    print(f'Minimum wavelength for crystal 1: {int(wavelength)} nm.', RegisterStatusTypes(result))

wavelength_min1()

Minimum wavelength for crystal 1: 1680 nm. Unknown status


In [5]:
# select class

def power_monitor(self):    # CHECK
    """Readout from optional optical power monitors. Result as a percent."""
    result1 = registerReadU16(self.COMport, self.SELECT_devID, 0x10, -1)
    power1 = result1[1]/10

    result2 = registerReadU16(self.COMport, self.SELECT_devID, 0x11, -1)
    power2 = result2[1]/10

    print(f'Power monitor 1 readout: {power1} %, power monitor 2 readout: {power2} %.')

In [ ]:
def monitor_switch(self, crystal=None):  # CHECK
    """Select which power detector should be connected to the output connector. Optional input 1 or 2 to switch to the corresponding crystal."""
    if crystal==None:
        result = registerReadU8(self.COMport, self.SELECT_devID, 0x35, -1)
        result = result[1]
        if result == 0:
            print('Optical power monitor connected to crystal 1.')
        elif result == 1:
            print('Optical power monitor connected to crystal 2.')

    elif crystal == 1:
        result = registerWriteReadU8(self.COMport, self.SELECT_devID, 0x35, 0, -1)
        print('Optical power monitor connected to crystal 1.')
    elif crystal == 2:
        result = registerWriteReadU8(self.COMport, self.SELECT_devID, 0x35, 1, -1)
        print('Optical power monitor connected to crystal 2.')

    else:
        print('Invalid input. Choose 1 or 2.')